In [26]:
import openpyxl
import pandas as pd
from module.analiz_data import *
import shutil
from openpyxl.utils.cell import coordinate_from_string
from openpyxl.utils import column_index_from_string
import os
import sys
from tkinter.filedialog import askopenfilename, asksaveasfilename
from tkinter import messagebox
from tkinter import Tk
Tk().withdraw()

# from openpyxl.utils import get_column_letter
# from copy import copy
# from openpyxl.styles import Font

''

In [27]:
def coordinate (cell):
    """Конвртер координат: A10 --- 10, 1"""
    data = coordinate_from_string (cell)
    row = data[1]
    col = column_index_from_string (data[0])
    return row, col 

In [28]:
def tabl_1_size (ws):
    """ Размер шапки таблицы в таблице XBRL"""
    col_size = None
    row_start = 5
    col_start = 1
    for col in range (col_start, ws.max_column + 1):
        if "Z=" in str (ws.cell(row_start, col).value):
            col_size = col
            break
    if col_size is None:        
        ERRORS.append(f'Размер таблицы в "{ws.title}" не найден')
        raise 
    return ws.max_row, col_size  

In [29]:
def end_col_number (ws, col_size, row_id_fond = 5 ):
    """ найти цифру (номер) в последней колонки в таблице XBRL
     и номер строки с идектификаторами """    
    # координаты ячейки с id фонда
    # row_id_fond = 5
    # col_size = 12
    
    # номер последней колонки
    for row in range (1, 5):        
        cell = ws.cell (row_id_fond + row, col_size)
        if cell.value and cell.value.isdigit():
            max_number = int(cell.value)            
            break
    return max_number, row_id_fond + row + 1 # строка с идентификаторами ниже на одну строчку


In [30]:
def begin_cell (ws, col_size, max_number, row_id_fond = 5 ):
    """ координаты первой ячейки в таблице XBRL """
    # row_id_fond = 5 - номер строки, в которой расположен "id_fond" 
    # max_number - цифра в ячейке правой крайней колонки
    # col_size - номер правой крайней колонки
    
    col_begin = col_size - (max_number - 1)
    for row in range (3,10):
        cell = ws.cell (row_id_fond + row, col_begin)        
        if cell.fill.start_color.index == '00000000':
            row_begin = row_id_fond + row
            break
    return col_begin, row_begin

In [31]:
def id_nombers (col_begin):
    """ номера колонок с иденитификаторами в таблице XBRL"""
    # координаты ячейки с id фонда
    # row_id_fond = 5
    # col_size = 12
    
    # номер колонкс с "Итого"
    col_itogo = 2
    # количество колонок с идентификаторами
    id_numbers = col_begin - col_itogo
    
    # номера колонок с идентификаторами
    id_cols = [ (col_itogo + col) for col in range (id_numbers)]
        
    return id_cols


In [32]:
def find_col_id_fond (ws, col_size, id_fond, row_id_fond = 5 ):
    """ Найти колонку с идентификатором фонда в таблице XBRL"""
    col_id_fond = False    
    for col in range (col_size, ws.max_row + 1, col_size):
        print ('...', col, ws.cell(row_id_fond, col).value)
        cell = ws.cell(row_id_fond, col).value
        if cell.endswith(id_fond):
            col_id_fond = col
            
    return col_id_fond


In [33]:
def razdel_name_row (df_avancor, title_1_name, index_max, title_col = 2):    
    """найти Номер строки с названием раздела в файле Аванкор"""
        
    for row in range(1, index_max):
        title = str (df_avancor.loc[row, title_col])
        if title == title_1_name or title[:20] == title_1_name[:20] :
            title_row = row
            return title_row

    ERRORS.append(f'Раздел отчетности: "{title_1_name}" в таблице-Аванкор не найден')
    raise 
        
    


In [34]:
def start_data_row (df_avancor, index_max, title_row, title_col = 2):
    """находим номер первой строку с данными в файле Аванкор""" 
    for row in range (1, index_max):
        data_i = df_avancor.loc[title_row + row, title_col]
        if str (data_i) == '1':
            data_row = title_row + row + 1
            return data_row

In [35]:
def end_data_row (df_avancor, index_max, data_row, title_col = 2):
    """находим номер последнке строки с данными в таблице Аванкор"""
    
    data = df_avancor.loc[data_row, title_col]
    if data == 'Итого' or str (data) == 'nan' or str(data).startswith ('Оценочная стоимость'):
        print (' - раздел пуст')    
        row_end = data_row
    else:        
        for row in range (1, index_max):
            data_i = df_avancor.loc[data_row + row, title_col]
            if data_i == 'Итого' or str (data_i) == 'nan':                
                row_end = data_row + row
                break
                    
    return row_end  


In [36]:
def find_columns_numbers (df_avancor, collumn_max, max_number, data_row, data_col = 3 ):
    """ Номера колонок в таблице Аванкор, за исключением пустых"""
    # data_row, data_col_1 - координаты первой колонки с данными в Аванкор
    # max_number - номер последней колонки в таблице XBRL
    
    column = []
    for i in range(0, max_number):                   
        while str (df_avancor.loc[data_row - 1, data_col]) == 'nan' and data_col <= collumn_max :
            data_col += 1
        column.append(data_col)
        data_col += 1
    return column


In [37]:
def copy_data (ws, df_avancor, rows_numbers, columns_numbers, row_begin, col_begin):
    """Копирование данных из таблицы Аванков в таблицу XBRL """
         
    for r, row in enumerate (rows_numbers):
        for c, col in enumerate (columns_numbers):
            cell_avancor = df_avancor.loc[row, col]
            if str (cell_avancor).startswith("Не установлен"):
                ERRORS.append (f'"{ws.title}"; '
                               f'строка: {row_begin + r}, '
                               f'колонка: {c+1} ; '
                               f'параметр: "{cell_avancor}"' )
            if cell_avancor != '-' and cell_avancor != "x" and cell_avancor != 0:          
                ws.cell(row_begin + r, col_begin + c).value = \
                    analiz_data_all (df_avancor.loc[row, col] )
                


In [38]:
def find_id (ws, df_id_ws, row_i, id_i1, id_fond):
    """ найти нужный идентификатор, анализируя содержание ячеек в строке таблицы XBRL"""
    # id_i1 - первый признак идентификатора
    # id_i2 - второй признак идентификатора
    
    # список найденных идентификаторов (первая колонка) 
    id_list = df_id_ws[ df_id_ws[1] == id_i1 ][0].to_list()  
    
    id_i2 = False # второй признак идентификатора
        
    # если список состоит из одного элемента, то идентификатор определен 
    if len (id_list) == 1:        
        id_name = id_list[0]
    else: # идентификатор НЕ определен
        # список вторых признаков идентификатора (третья колонка)
        id_str_2 = df_id_ws[df_id_ws[1] == id_i1 ][2].to_list()
                
        # просматриваем ячейки в строке
        for col_i in range (col_begin, col_size + 1 ):
            # Признак идентификатора
            id_i2 = ws.cell(row_i, col_i).value
            
            if id_i2 in id_str_2: # в строке есть второй признак
                break        
        if not id_i2: # если в строке второй признак не найден, то находим связь с "id_fond"            
            id_i2 = id_fond
            
        id_list = df_id_ws[(df_id_ws[1] == id_i1) & (df_id_ws[2] == id_i2)][0].to_list()
        id_name = id_list[0] 
        
    return id_name


def insert_id (ws, id_row, id_cols, df_id, rows_numbers, row_begin, col_begin, id_fond ):   
    """ Находим и вставляем идентификаторы в ячейки"""
    
    # словарь ВСЕХ названий идетификаторов (полные) и имен вкладок (сокращенные)
    list_id = {(df_id[k].loc[0,0]): k for k in df_id.keys() }
    
    # список идентификаторов на листе 
    id_on_list = [ws.cell(id_row, col).value for col in id_cols ]
    # находим признаки идентификатора в строке
    for i, id_name in enumerate (id_on_list): # полные названия идентификаторов    
        # название листа в файле с идентификатрами 
        sheet_name_id = list_id [id_name]
        # открываем нужную страницу в файле с идентификаторами
        df_id_ws = df_id[sheet_name_id]
                
        # находим множество всех признаков идентификаторов (исключая нулевую строку)
        # вторая колонка
        id_n = set (df_id_ws [1].to_list()[1:])
                
        for row in range (len (rows_numbers) - 1):
            row_i = row_begin + row
            id_on = False
            for col in range (max_number):
                                
                col_i = col_begin + col 
                # Признак идентификатора
                id_i = ws.cell(row_i, col_i).value        
                
                # Если признак состоит из одних цифр, то преобразуем его в целое число, 
                # т.к. это либо ИНН либо ОГРН               
                if type(id_i) == str and id_i.isdigit():
                    id_i = int (id_i)    
                                
                # если находим признак идентификатора, то прерываем пеербор колонок
                if id_i in id_n:
                    id_on = True
                    # записываем название идентификатора в таблицу xbrl                 
                    ws.cell(row_i, id_cols[i]).value = find_id (ws, df_id_ws, row_i, id_i, id_fond) 
                    break
            
            # исусственное установление признака "Наименование биржи" == 'NoName', 
            # так как при указании не внебиржевые бумаги поле в отчетностит остается пустым               
            if 'NoName' in id_n:
                id_on = True
                id_i = 'NoName'
                ws.cell(row_i, id_cols[i]).value = find_id (ws, df_id_ws, row_i, id_i, id_fond) 
            
            if not id_on:                
                ERRORS.append(f'"{ws.title}"; Идентификатор - "{sheet_name_id}" -  не найден!')

In [39]:
def copy_id_fond_to_tbl (wb, id_fond): 
    """Записываем во все формы идентификатор фонда"""
    
    # Выбираем вкладки, содержащие идентификатор фонда
    sheet_2_with_id_fond = wb.sheetnames
    sheet_2_with_id_fond.remove('0420502 Справка о стоимости чис')
    sheet_2_with_id_fond.remove('0420502 Справка о стоимости _56')
    sheet_2_with_id_fond.remove('0420502 Справка о стоимости _57')
    sheet_2_with_id_fond.remove('_dropDownSheet')
    
    # Записываем в выбранные вкладки идентификатор фонда
    for form in sheet_2_with_id_fond:    
        ws = wb[form]
        # Размер шапки таблицы таблицы XBRL
        row_size, col_size = tabl_1_size(ws)
        # записывает во все таблицы xbrl идентификатор фонда
        ws.cell(row = 5, column = col_size ).value = 'Z= Идентификатор АИФ ПИФ-' + id_fond 

In [40]:
def load_matrica ():
    """загружвем данные из матрицы"""
    file_matrica = r'./Шаблоны/Матрица_0420502.xlsx'
    df_matrica = pd.read_excel(file_matrica, sheet_name='0420502', index_col=1)
    #df_matrica.head(3)
    return df_matrica

In [41]:
def load_avancor (file_avancor):
    # загружвем данные из отчета таблицы Аванкор

    df_avancor = pd.read_excel(file_avancor, sheet_name='TDSheet', header=None)
    
    # устанавливаем начальный индекс не c 0, а c 1  
    df_avancor.index += 1
    df_avancor.columns += 1
    
    return df_avancor

In [42]:
def copy_fio(wb_xbrl, df_matrica, df_avancor):

    # отбор форм подписантов
    sheet_2_fio = wb_xbrl.sheetnames[61:63]
    
    for form in sheet_2_fio: 
        # находим, используя матрицу, раздел в файле Аванкор, соответствующий выбранной форме 
        title_1_name = df_matrica.loc[form,'sheet_1_title']
        
        # количество строк в таблице Аванкор
        index_max = df_avancor.shape[0]
        # Номер строки с названием раздела в файле Аванкор           
        title_row = razdel_name_row (df_avancor, title_1_name, index_max)
        
        # номер колонки ячейки с данными в таблице Аванкор
        data_col = df_matrica.loc[form, 'cell1_col']
        data_col = column_index_from_string (data_col)
        
        # ФИО подписанта
        fio = df_avancor.loc[title_row, data_col]
        
        # записываем в форму xbrl
        ws_fio = wb_xbrl[form]
        row_fio, col_fio = coordinate(df_matrica.loc[form,'cell_fio'])
        ws_fio.cell(row_fio, col_fio).value = fio
        
        print (ws_fio.cell(row_fio, col_fio).value)

In [43]:
def copy_period (wb_xbrl, df_avancor, df_matrica):
    """ Вставляем данные о периоде отчетности """
    period_begin = analiz_data_data (df_avancor.loc [18,1])
    period_end = analiz_data_data (df_avancor.loc [18,5])
        
    sheet_xbrl_period = wb_xbrl.sheetnames[8:18]
    for sheet in sheet_xbrl_period:
        ws_period = wb_xbrl[sheet]
        cell_period = df_matrica.loc[sheet, 'cell_period']
        ws_period[cell_period].value = period_begin + ', ' + period_end
        

In [44]:
def copy_pai (id_fond, wb_xbrl, df_avancor, df_id):
    """ Копируем количество паев с нужной точностью"""
    
    # Точность указания кол-ва паев
    df_id_ws = df_id['ПИФ']
    fix = df_id_ws[df_id_ws[0] == id_fond ][3].values[0]
    
    # записываем в форму 
    ws_pai = wb_xbrl['0420502 Справка о стоимости _13']
    today_row, today_col = coordinate('I144')
    previous_row, previous_col = coordinate('K144')
    
    today = toFixed(df_avancor.loc[today_row, today_col], digits=fix)
    previous = toFixed (df_avancor.loc [previous_row, previous_col], digits=fix) 
    
    ws_pai.cell(10,3).value = today
    ws_pai.cell(10,4).value = previous
       

In [45]:
def write_errors ():
    """ Записываем ошибки в файл"""
    if not ERRORS:
        ERRORS[0] = 'Ошибок не выявлено!'
    with open(errors_file, "w") as file:
        for k in ERRORS:
            file.write(str(k) + '\n\n')

In [46]:
def file_open(df_id):
    """ Выбор файла, созданного Аванкор"""
    
    # Список всех идентификаторой фондов
    all_id_fond = df_id['ПИФ'][0][1:].to_list()
    
    # Выбираем файл, сформированный Аванкор
    file_open = askopenfilename() # show an "Open" dialog box and return the path to the selected file
    # Имя файла без пути к нему
    file_avancor = os.path.basename(file_open) 
    # Имя файла без расширения (идентификатор фонда)
    id_fond = os.path.splitext(file_avancor)[0]
    
    # Если в названии файла нет идентификатора, то прерываем программу 
    if not (id_fond in all_id_fond ):
        ERRORS.append(f'{"="*100}\n'
                      f'Файл не сформирован!\n'
                      f'В названии файла: "{file_avancor}" неверно указан идентификатор фонда!\n'
                      f'(проверьте название файла)\n'
                      f'{"="*100}')
        write_errors ()
        sys.exit("Ошибка в имени файла!")
    
    return file_avancor, id_fond


In [47]:
############################################################################################

In [48]:
# файл с ощибками
errors_file = 'errors.txt'
ERRORS = []

# # Если нет файла, то создаем пустой файл
# if not os.path.exists(errors_file):
#     with open(errors_file, "w") as F1:
#         pass
#--------------------------------------------
# загружаем данные из файла c идентификаторами
file_id = r'./Шаблоны/Идентификаторы.xlsx'
df_id = pd.read_excel(file_id, sheet_name=None, header=None)

In [49]:
 # Выбор файла, созданного Аванкор
file_avancor, id_fond = file_open(df_id)
#--------------------------------------------

In [50]:
# добавляем к названию файла ошибок идентификатор фонда
errors_file = os.path.splitext(errors_file)[0]   + " - " \
              + os.path.splitext(file_avancor)[0] \
              + os.path.splitext(errors_file)[1] 
#--------------------------------------------

In [51]:
# загружвем данные из отчета таблицы Аванкор
df_avancor = load_avancor(file_avancor)


In [52]:
# загружвем данные из матрицы
df_matrica = load_matrica ()

#--------------------------------------------

# кол-во строк и столбцов в файле Аванкор
index_max = df_avancor.shape[0]
collumn_max = df_avancor.shape[1]
#df_avancor.head(3)
#--------------------------------------------
# # Определяем идентификатор фонда их имени файла Аванкор
# id_fond = file_avancor.split('.')[0]
#--------------------------------------------
# название нового файла-отчетности xbrl
file_fond_name = asksaveasfilename()
# Добавляем расширение файла
file_fond_name = file_fond_name + '.xlsx'
# отбрасываем путь к файлу
file_fond = os.path.basename(file_fond_name)
# file_fond = id_fond + '.xlsx'

#--------------------------------------------
# название файла-шаблона
file_shablon = '0420502(шаблон)(new).xlsx'
#--------------------------------------------
# Создаем новый файл отчетности xbrl из шаблона
shutil.copyfile(r'./Шаблоны/'+file_shablon, r'./'+file_fond  )


# Загружаем данные из файла таблицы xbrl
wb = openpyxl.load_workbook(filename=file_fond) 
#wb.sheetnames
#--------------------------------------------
# Записываем во все формы идентификатор фонда
copy_id_fond_to_tbl (wb, id_fond)
#--------------------------------------------    

# Копируем данные расшифровки
# Выбираем формы: поясн.записки и расшифровки разделов
sheet_2_decryption_names = wb.sheetnames[19:]
sheet_2_decryption_names.remove('0420502 Справка о стоимости _56')
sheet_2_decryption_names.remove('0420502 Справка о стоимости _57')
sheet_2_decryption_names.remove('_dropDownSheet')

form_null = [] # список пустых форм
for form in sheet_2_decryption_names:
    print (f'... {form}')
  
    ws = wb[form]
    
    # Размер шапки таблицы таблицы XBRL
    row_size, col_size = tabl_1_size(ws)
    
    # находим, используя матрицу, раздел в файле Аванкор, соответствующий выбранной форме 
    title_1_name = df_matrica.loc[form,'sheet_1_title']
    
    # Номер строки с названием раздела в файле Аванкор"""           
    title_row = razdel_name_row (df_avancor, title_1_name, index_max)
        
    # находим номер первой строку с данными в файле Аванкор
    data_row = start_data_row (df_avancor, index_max, title_row)
    
    # находим номер последнке строки с данными в таблице Аванкор"""
    row_end = end_data_row (df_avancor, index_max, data_row)
    
    
    if data_row != row_end:
        # список всех номеров строк в таблице Аванкор
        rows_numbers = [x for x in range (data_row,row_end + 1)]   
    else:
        # если номера первой и последней строки совпадают, то Раздел пуст и
        # запоминаем название вкладки
        form_null.append (form)
        # переходим к поиску следующего раздела
        continue
   
    # цифра в последней колонке и номер строки с идектификаторами в таблице XBRL
    # (количество колонок для копирования)
    max_number, id_row = end_col_number (ws, col_size)
    
    # Вставить нужное кол-во строк перед строкой "Итого"
    ws.insert_rows(row_size, amount= (len (rows_numbers) - 1))
    
    # координаты первой ячейки в таблице XBRL
    col_begin, row_begin = begin_cell (ws, col_size, max_number)
    
    # номера колонок с иденитификаторами в таблице XBRL
    id_cols = id_nombers (col_begin)
           
    # Номера колонок в таблице Аванкор, за исключением пустых"""
    columns_numbers = find_columns_numbers(df_avancor, collumn_max, max_number, data_row )
    
    # Копирование данных из таблицы Аванков в таблицу XBRL 
    copy_data (ws, df_avancor, rows_numbers, columns_numbers, row_begin, col_begin)                        
    
    # Копирование идентификаторов
    # Находим и вставляем идентификаторы в ячейки
    insert_id (ws, id_row, id_cols, df_id, rows_numbers, row_begin, col_begin, id_fond )
    
    print (f' - ОК!')
    
    # Сохраняем в файл отчетности xbrl
    wb.save(file_fond)

c:\users\сотрудник\yandexdisk-atovanchov\projects\excel_02\venv\lib\site-packages\openpyxl\worksheet\_reader.py:296: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


... 0420502 Справка о стоимости _14
 - ОК!
... 0420502 Справка о стоимости _15
 - раздел пуст
... 0420502 Справка о стоимости _16
 - раздел пуст
... 0420502 Справка о стоимости _17
 - раздел пуст
... 0420502 Справка о стоимости _18
 - раздел пуст
... 0420502 Справка о стоимости _19
 - раздел пуст
... 0420502 Справка о стоимости _20
 - раздел пуст
... 0420502 Справка о стоимости _21
 - раздел пуст
... 0420502 Справка о стоимости _22
 - раздел пуст
... 0420502 Справка о стоимости _23
 - раздел пуст
... 0420502 Справка о стоимости _24
 - ОК!
... 0420502 Справка о стоимости _25
 - раздел пуст
... 0420502 Справка о стоимости _26
 - раздел пуст
... 0420502 Справка о стоимости _27
 - раздел пуст
... 0420502 Справка о стоимости _28
 - раздел пуст
... 0420502 Справка о стоимости _29
 - раздел пуст
... 0420502 Справка о стоимости _30
 - раздел пуст
... 0420502 Справка о стоимости _31
 - раздел пуст
... 0420502 Справка о стоимости _32
 - раздел пуст
... 0420502 Справка о стоимости _33
 - раздел п

In [53]:
# Копируем итоговые данные
sheet_2_itigi = wb.sheetnames[6:19]

for form in sheet_2_itigi:
       
    print (form)
    # находим, используя матрицу, раздел в файле Аванкор, соответствующий выбранной форме 
    title_1_name = df_matrica.loc[form,'sheet_1_title']
    
    # координаты первой ячейки с данными в таблице Аванкор
    cell_start = df_matrica.loc[form,'cell1']    
    cell_start_row, cell_start_col = coordinate(cell_start)
    
    # координаты последней ячейки с данными в таблице Аванкор
    cell_end = df_matrica.loc[form,'cell_end']
    cell_end_row, cell_end_col = coordinate(cell_end)
        
    # (количество колонок для копирования) в таблице Аванкор
    max_number = df_matrica.loc[form,'tbl_col']
      
    # Номера колонок в таблице Аванкор, за исключением пустых
    columns_numbers = find_columns_numbers(df_avancor, cell_end_col, max_number, cell_start_row, data_col = cell_start_col)
    print (title_1_name, columns_numbers)
    
    # список всех номеров строк в таблице Аванкор
    rows_numbers = [x for x in range (cell_start_row,cell_end_row + 1)] 
    
    ws = wb[form]
    qq = ws.max_column
    # координаты первой ячейки в таблице xbrl
    col_begin, row_begin = begin_cell (ws, ws.max_column, max_number)
    
    # Копирование данных из таблицы Аванков в таблицу XBRL 
    copy_data (ws, df_avancor, rows_numbers, columns_numbers, row_begin, col_begin)



0420502 Справка о стоимости чис
Раздел I. Реквизиты акционерного инвестиционного фонда (паевого инвестиционного фонда) [1, 5, 7, 11, 13]
0420502 Справка о стоимости ч_2
Раздел II. Параметры справки о стоимости чистых активов [1, 5, 7]
0420502 Справка о стоимости ч_3
Подраздел 1. Денежные средства [9, 11, 13, 15]
0420502 Справка о стоимости ч_4
Подраздел 4. Недвижимое имущество и права аренды недвижимого имущества [9, 11, 13, 15]
0420502 Справка о стоимости ч_5
Подраздел 2. Ценные бумаги российских эмитентов (за исключением закладных) [9, 11, 13, 15]
0420502 Справка о стоимости ч_6
Подраздел 3. Ценные бумаги иностранных эмитентов [9, 11, 13, 15]
0420502 Справка о стоимости ч_7
Подраздел 5. Имущественные права (за исключением прав аренды недвижимого имущества, прав из кредитных договоров и договоров займа и прав требования к кредитной организации выплатить денежный эквивалент драгоценных металлов) [9, 11, 13, 15]
0420502 Справка о стоимости ч_8
Подраздел 6. Денежные требования по кредитн

In [54]:
# Копируем данные из пояснительных записок
sheet_2_zapiski_names = wb.sheetnames[:6]
# удаляем лишнюю форму (этой формы нет в Аванкоре)
sheet_2_zapiski_names.remove('0420502 Пояснительная записка_5')


form_zapiski_null = [] # список пустых форм
for form in sheet_2_zapiski_names:
    print (f'... {form}')
  
    ws = wb[form]
    
    # Размер шапки таблицы таблицы XBRL
    row_size, col_size = tabl_1_size(ws)
    
    # находим, используя матрицу, раздел в файле Аванкор, соответствующий выбранной форме 
    title_1_name = df_matrica.loc[form,'sheet_1_title']
    
    # Номер строки с названием раздела в файле Аванкор"""           
    title_row = razdel_name_row (df_avancor, title_1_name, index_max)
        
    # находим номер первой строку с данными в файле Аванкор
    data_row = start_data_row (df_avancor, index_max, title_row)
    
    # находим номер последней строки с данными в таблице Аванкор"""
    row_end = end_data_row (df_avancor, index_max, data_row)
    
    
    if df_avancor.loc[row_end - 1, 3] != '2': 
        # список всех номеров строк в таблице Аванкор
        rows_numbers = [x for x in range (data_row,row_end)]   
    else:        
        # Если в соседней ячейки(колонка "C") == "2", то это заголовок таблицы и следовательно форма пустая
        # запоминаем название вкладки
        form_zapiski_null.append (form)
        # переходим к поиску следующего раздела
        continue
    
    # цифра в последней колонке и номер строки с идектификаторами в таблице XBRL
    # (количество колонок для копирования)
    max_number = df_matrica.loc[form, 'tbl_col']
    # max_number, id_row = end_col_number (ws, col_size)
    
    # координаты первой ячейки в таблице XBRL
    cell_start = df_matrica.loc[form,'cell2']    
    cell_start_row, cell_start_col = coordinate(cell_start)
    
    # Номера колонок в таблице Аванкор, за исключением пустых
    columns_numbers = find_columns_numbers(df_avancor, collumn_max, max_number, data_row, data_col = 3)
    print (title_1_name, columns_numbers)
    
    # Копирование данных из таблицы Аванков в таблицу XBRL 
    copy_data (ws, df_avancor, rows_numbers, columns_numbers, cell_start_row, cell_start_col)  
    ERRORS.append (f'"{ws.title}" - вставьте "Идентификатор строки"')

... 0420502 Пояснительная записка к
 - раздел пуст
... 0420502 Пояснительная записка_2
Сведения о событиях, которые оказали существенное влияние на стоимость чистых активов акционерного инвестиционного фонда или чистых активов паевого инвестиционного фонда [3]
... 0420502 Пояснительная записка_3
 - раздел пуст
... 0420502 Пояснительная записка_4
 - раздел пуст
... 0420502 Пояснительная записка_6
 - раздел пуст


In [55]:
# Ручная корректировка

# меняем значения ячеек, т.к. в таблице xbrl ячейки расположены в другом порядке
ws_change = wb['0420502 Справка о стоимости чис']    

ws_change.cell(10,5).value, ws_change.cell(10,6).value =\
ws_change.cell(10,6).value, ws_change.cell(10,5).value       

ws_change.cell(10,5).value, ws_change.cell(10,7).value =\
ws_change.cell(10,7).value, ws_change.cell(10,5).value   

# записываем в форму идентификатор фонда
ws_id = wb['0420502 Справка о стоимости _57']
ws_id.cell(8,1).value = id_fond

# записываем в форму идентификаторы: фонда и УК
ws_id = wb['0420502 Справка о стоимости чис']
ws_id.cell(10,1).value = id_fond
ws_id.cell(10,2).value = df_id['УК АИФ ПИФ'].loc[1,0]


# ФИО подписанта СпецДепа из файла идентификаторов
# df_id_ws = df_id['ПИФ']
# fio = df_id_ws[df_id_ws[0] == id_fond ][2].values[0]

# Записываем в форму ФИО подписантов
copy_fio(wb, df_matrica, df_avancor)

# Вставляем данные о периоде отчетности
copy_period (wb, df_avancor, df_matrica)

# Копируем кол-во паев с заданной точностью
copy_pai (id_fond, wb, df_avancor, df_id)

А.Я.Тованчов
Г.Н.Панкратова


In [56]:
# удаляем пустые формы
for name in (form_null + form_zapiski_null): 
    wb.remove(wb[name])
wb.remove(wb['0420502 Пояснительная записка_5'])

In [57]:
# Сохраняем в файл отчетности xbrl
try: 
    # os.chdir (os.path.dirname (file_fond_name))
    wb.save(file_fond)
    print('-------------------ГОТОВО!!!----------------------')
    
    messagebox.showinfo("Конвертер", "Файл создан!")
    
    
    
except PermissionError:
    ERRORS.append(f'{"="*100}\n'
                  f'!!!ФАЙЛ НЕ СОЗДАН!!! \n'
                  f'ОШИБКА ДОСТУПА К ФАЙЛУ '
                  f'(файл открыт в другой программе - закройте файл!)\n'
                  f'{"="*100} ')

-------------------ГОТОВО!!!----------------------


In [58]:
# Записываем ошибки
write_errors ()
    

In [59]:
#
# from importlib import reload
# import Modules.analiz_data
# reload(Modules.analiz_data)
# from Modules.analiz_data import *

# import sys
# from .application.app.Modules.analiz_data import *


In [60]:
# ошибка - "Не установлена"